In [1]:
# Import modules

import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import time
import pandas as pd
from tabulate import tabulate
from tqdm import tqdm

In [2]:
# p(s|t), used in Q.4-Q.6
def p_st(mu_s, covar_s, covar_ts, A, ATranspose, t):
    
    mu_sTranspose = mu_s[:,None]
    covar_sInv = np.linalg.inv(covar_s)
    
    covar_st = np.linalg.inv(covar_sInv + (ATranspose * (1/covar_ts) * A))
    mu_st = covar_st @ (covar_sInv @ mu_sTranspose + ATranspose * (1/covar_ts) *t)
    
    return stats.multivariate_normal.rvs(np.ravel(mu_st), covar_st)

# p(t|s), truncated gaussian, used in Q.4-Q.6
def p_ts(y, s1, s2, covar_s, covar_ts, A, ATranspose):
    
    # Player 1 wins
    if y == 1:
        ing1 = 0
        ing2 = np.Inf
        
    # Player 2 wins
    else:
        ing1 = -np.Inf
        ing2 = 0
    
    mu_t = s1 - s2
    sigma_t  = np.sqrt(covar_ts +A @ covar_s @ ATranspose)
    
    return stats.truncnorm.rvs((ing1 - mu_t) / sigma_t, (ing2 - mu_t) / sigma_t, loc=mu_t, scale=sigma_t)

# p(s), used in Q.4
def p_s(x, mu, sigma):
    
    return stats.norm.pdf(x, mu, sigma)


In [3]:
# Gibbs sampler, used in Q.4-Q.6
def gibbs(K, mu_s, cov_s, cov_ts, t0, burn_in=0, y=1):
    
    A = np.array([1, -1])
    ATranspose = A[:,None]
    
    s1 = np.zeros(K)
    s2 = np.zeros(K)
    t = np.zeros(K)
    
    s1[0] = mu_s[0] 
    s2[0] = mu_s[1]
    t[0] = t0

    for k in range(K - 1):
        
        s1[k+1], s2[k+1] = p_st(mu_s, covar_s, covar_ts, A, ATranspose, t[k])
        t[k+1] = p_ts(y, s1[k+1], s2[k+1], covar_s, covar_ts, A, ATranspose)
        
    return s1[burn_in::], s2[burn_in::], t[burn_in::]

In [4]:
# Moment-matching, used in Q.8

def moment_matching():
    

SyntaxError: incomplete input (2081212655.py, line 4)

In [ ]:
def trueskillGaussDist(X, s1, s2):
    
    plt.plot(X, p_s(X, m_s2, var_s2), label="p(s1) and p(s2)", linestyle="dashed", color="orange")
    plt.plot(X, p_s(X, np.mean(s1), np.std(s1)), label="p(s1|y=1)", color="blue")
    plt.plot(X, p_s(X, np.mean(s2),np.std(s2)), label="p(s2|y=1)", color="purple")
    plt.legend()
    plt.show()

In [ ]:
Klist = [250, 500, 750, 1000, 2500, 5000]
#Klist = [700, 750, 800, 850]
burn_in = 40
def performance_check(Kl, mu_s, covar_s, covar_ts, t0, burn_in):
    
    x = np.linspace(0, 50, 1000)
    for k in Kl:
        
        t_start = time.perf_counter()
        s1, s2, t = gibbs(k+burn_in, mu_s, covar_s, covar_ts, t0, burn_in)
        t_stop = time.perf_counter()
        
        plt.plot(x, p_s(x, np.mean(s1), np.std(s1)), label="S1 post", color="red")
        plt.hist(s1,bins=30, density=True)
        plt.title("K = {}, time = {:.2f}".format(k, (t_stop - t_start)))
        plt.show()
        


In [ ]:
def dataPreprocesser(df, dataset='default'):
    
    # Create a new column for the result and filter the draws
    if dataset=='default':
        df['result'] = np.sign(df["score1"] - df["score2"])
    elif dataset=='chess':
        df.columns = ['team1','team2' ,'result']
        df['result']=chess_df['result'].replace('1/2-1/2', 0).replace('1-0', 1).replace('0-1', -1)
    df = df.loc[df['result'] != 0]
    
    df2 = pd.DataFrame()
    df2["club"] = df["team1"].append(df["team2"]).unique()
    df2["win"], df2["loss"], df2["mean"], df2["variance"] = 0, 0, 25, 25/3
    
    for i, row in df.iterrows():
        if row["result"] == 1:
            df2.loc[df2["club"] == row["team1"], "win"] += 1
            df2.loc[df2["club"] == row["team2"], "loss"] += 1
        elif row["result"] == -1:
            df2.loc[df2["club"] == row["team2"], "win"] += 1
            df2.loc[df2["club"] == row["team1"], "loss"] += 1
        else:
            print("Dataframe contains draws.")

    return df, df2
    


In [ ]:
# Q.4 Gibbs sampler, one case where y = 1

# Same hyperparameters for p(s1) and p(s2)
# Default hyperparameter values are from "trueskill.org"
m_s1 = 25
m_s2 = 25
mu_s = np.array([m_s1, m_s2])
var_s1 = 25/3
var_s2 = 25/3
covar_s = np.array([[var_s1, 0], [0, var_s2]])
covar_ts = 5
t0 = 3

# Set number of iterations
K = 1000

# Q.4 a) Burn in = 40
burn_in = 40

# Perform Gibbs sampling (y=1)
s1, s2, t = gibbs(K, mu_s, covar_s, covar_ts, t0, burn_in, 1)


# plot the result
kx = np.linspace(0, K-burn_in,K-burn_in)
plt.plot(kx, s1, label="s1 samples")
plt.plot(kx, s2, label="s2 samples")
plt.plot(kx, t, label="t samples")
plt.legend()
plt.show()
print(f'S1 -> Mean: {np.mean(s1)}, Variance: {np.var(s1)}\nS2 -> Mean: {np.mean(s2)}, Variance: {np.var(s2)}')

# Q.4 b) and d) They look good. p(s1) have an increased mean while the opposite happened to p(s2), this is because
# the initial condition was that y=1, player 1 won, thus that players skill level must increase more than s2.
xx = np.linspace(0, 50, 1000)    
trueskillGaussDist(xx ,s1, s2)

# Q.4 c) k= 750 was choosen because most of the sample values were in the Gaussian distribution and still had
# a good computational time, increasing the k to 1000 didn't make an enough of a difference in accuracy to justify the
# increase int computational time.
performance_check(Klist, mu_s, covar_s, covar_ts, 10, burn_in)

In [ ]:
# Used for Q.5 and Q.6

def assume_density(df, K, burn_in, shuffle=False, one_step_predict=False, dataset='default'):
    
    # Preprocess the dataframe
    df, clublist = dataPreprocesser(df, dataset)
    
    # Shuffle the data randomly (if toggled on)
    if shuffle:
        df = df.sample(frac=1, random_state=np.random.RandomState())
        
    # Initiate variables (if toggled on)
    total_predictions = df.shape[0]
    correct_prediction = 0

    # Iterate through all matches
    for i, row in tqdm(df.iterrows()):
 
        # Set hyperparameters (default values from trueskill.org)
        m_s1 = float(clublist.loc[clublist["club"] == row["team1"],"mean"])
        m_s2 = float(clublist.loc[clublist["club"] == row["team2"],"mean"])
        mu_s = np.array([m_s1, m_s2])  
        var_s1 = float(clublist.loc[clublist["club"] == row["team1"],"variance"])
        var_s2 = float(clublist.loc[clublist["club"] == row["team2"],"variance"])
        covar_s = np.array([[var_s1, 0], [0, var_s2]])
        covar_ts = 25/3
        t0 = 30
        
        # Predict result (if toggled on)
        if one_step_predict:
            prediction = np.sign(m_s1 - m_s2)
            correct_prediction += (prediction == row["result"])
            
        # Perform Gibbs sampling
        s1, s2, t = gibbs(K, mu_s, covar_s, covar_ts, t0, burn_in, row["result"])
      
        # Update hyperparameters
        clublist.loc[clublist["club"] == row["team1"],"mean"] = np.mean(s1)
        clublist.loc[clublist["club"] == row["team2"],"mean"] = np.mean(s2)
        clublist.loc[clublist["club"] == row["team1"],"variance"] = np.var(s1)
        clublist.loc[clublist["club"] == row["team2"],"variance"] = np.var(s2)
    
    # Print results
    print(tabulate(clublist.sort_values(by="win", ascending=False), headers="keys", tablefmt="psql"))
    print(f'Correct predictions:\t{correct_prediction}\nTotal predictions:\t{total_predictions}\nPrediction rate (r):\t{correct_prediction / total_predictions}')




In [ ]:
df = pd.read_csv("SerieA.csv")
K = 750
burn_in = 40    
    
assume_density(df, K, burn_in, True, True)

In [ ]:
#

In [ ]:
# Task 9. Dataset of chess championship

chess_df  = pd.read_csv('chess_wc_history_game_info.csv')[['white', 'black', 'result']]
chess_df


In [ ]:
K = 750
burn_in = 40   
assume_density(chess_df, K, burn_in, True, True, dataset = 'chess')

